<a href="https://colab.research.google.com/github/QuickLearner171998/Object-Loc/blob/master/Object_Localisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [0]:
#Authorise colab

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
# to get files directly from drive
from google.colab import drive
drive.mount('gdrive')

 # To use this code as it is make the text files in the same format.

1) Make a train_labels.txt file which contains image name and image id .
     Format: id img_name
    
2) Make a train_coords.txt with format: id x1 y1 w h .
     x1,y1 are the top left corner, and  w, h are the width and height of the bounding box.
     
3) Create testid.txt which contains id and  test image names
     format: id img_name

# Preparing training dataset

You can use **LabelImg**. It is an open source software for drawing bounding boxes around the objects in images. From this you can easily get the coordinates of bounding boxes for train_coords.

#Get_data_train

In [0]:
from PIL import Image
import numpy as np
import pickle

def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

def Normalize(image,mean,std):
    for channel in range(3):
        image[:,:,channel]=(image[:,:,channel]-mean[channel])/std[channel]
    return image
  
  
# creating a custom data generator.  
def Imagegen(start,batchsize):
    id_to_data={}
    #id_to_size={}   
    #use id_to_size when the size of all images is not same.
    j=1
    i=start
    
    with open("train_labels.txt") as f:
        lines=f.read().splitlines()
        for line in lines:
            if (j>=i) and (j<i+batchsize):
                #id is the serial number of tha image starting  from 1
                id,path=line.split(" ",1)
                image=Image.open("train_images/"+path).convert('RGB')
                #id_to_size[int(id)]=np.array(image,dtype=np.float32).shape[0:2]                
                image=image.resize((224,224))
                image=np.array(image,dtype=np.float32)
                image=image/255
                image=Normalize(image,[0.485,0.456,0.406],[0.229,0.224,0.225])
                id_to_data[int(id)]=image                
                
            j=j+1
    id_to_data=np.array(list(id_to_data.values()))
    #id_to_size=np.array(list(id_to_size.values()))
    f=open("id_to_data","wb+")
    pickle.dump(id_to_data,f,protocol=4)
   # f=open("id_to_size","wb+")
   # pickle.dump(id_to_size,f,protocol=4)
    
    
   
    j=1
    k=1
    i=start
    width = 640
    height = 480

    id_to_box={}
    with open("train_coords.txt") as f:
        lines=f.read().splitlines()
        for line in lines:
            if (j>=i) and (j<i+batchsize):
                id,box=line.split(" ",1)
                box=np.array([float(i) for i in box.split(" ")],dtype=np.float32)
                box[0]=box[0]/width*224
                box[1]=box[1]/height*224
                box[2]=box[2]/width*224
                box[3]=box[3]/height*224
                
#                 if size of images different                
#                 box[0]=box[0]/id_to_size[int(id)-1][1]*224
#                 box[1]=box[1]/id_to_size[int(id)-1][0]*224
#                 box[2]=box[2]/id_to_size[int(id)-1][1]*224
#                 box[3]=box[3]/id_to_size[int(id)-1][0]*224

                id_to_box[int(k)]=box
                k=k+1
            j=j+1
    id_to_box=np.array(list(id_to_box.values()))
    f=open("id_to_box","wb+") 
    pickle.dump(id_to_box,f,protocol=4)
    return id_to_box,id_to_data
    
i=1
batchsize=1000
#number of total images = 24000
for i in range(1,24000,batchsize):
    id_to_box,id_to_data = Imagegen(i,batchsize)
    f=open("id_to_box","wb+") 
    pickle.dump(id_to_box,f,protocol=4)
    f=open("id_to_data","wb+")
    pickle.dump(id_to_data,f,protocol=4)
    with open("final_id_to_data", "ab") as myfile, open("id_to_data", "rb") as file2:
      myfile.write(file2.read())
      with open("final_id_to_box", "ab") as myfile, open("id_to_box", "rb") as file2:
      myfile.write(file2.read())
    f =open("final_id_to_box","ab")
    pickle.dump(id_to_box,f,protocol=4)
    f = open("final_id_to_data","ab")
    pickle.dump(id_to_data,f,protocol=4)




#Train

In [0]:

import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
import numpy as np
import keras.losses
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
import random

plt.switch_backend('agg')

# data: Train-23000, test-1000

def getdata():
  train_data = 23000
  test_data=1000
  btch_size =1000
  runs = train_data/btch_size
  tst_run = test_data/btch_size
 # read data and shuffle
  index=[i for i in range(test_size)]
  random.shuffle(index)

  
  f=open("final_id_to_data","rb")
  data = pickle.load(f)
  for j in range(0,runs):
    if j==(runs - tst_runs):
      data1 = pickle.load(f)
    elif j>(runs - tst_runs):
      data1 = np.concatenate((data1, pickle.load(f)), axis=0)
    else: 
      data = pickle.load(f)
  data_test=data1[index]
  
  
  f=open("final_id_to_box","rb")
  box=pickle.load(f)
  for j in range(0,runs):
    if j==(runs - tst_runs):
      box1 = pickle.load(f)
    elif j>(runs - tst_runs):
      box1 = np.concatenate((box1, pickle.load(f)), axis=0)
    else: 
      box = pickle.load(f)
  box_test=box1[index]
  return data_test,box_test

def plot_model(model_details):
    fig, axs = plt.subplots(1,2,figsize=(15,5))

    axs[0].plot(range(1,len(model_details.history['my_metric'])+1),model_details.history['my_metric'])
    axs[0].plot(range(1,len(model_details.history['val_my_metric'])+1),[1.7*x for x in model_details.history['val_my_metric']])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_details.history['my_metric'])+1),len(model_details.history['my_metric'])/10)
    axs[0].legend(['train', 'val'], loc='best')

    axs[1].plot(range(1,len(model_details.history['loss'])+1),model_details.history['loss'])
    axs[1].plot(range(1,len(model_details.history['val_loss'])+1),model_details.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_details.history['loss'])+1),len(model_details.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')

    plt.savefig("model.png")


    
# metric function
def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*224
    x=tf.maximum(tf.minimum(x,224.0),0.0)
    y=predictions[:,1]*224
    y=tf.maximum(tf.minimum(y,224.0),0.0)
    width=predictions[:,2]*224
    width=tf.maximum(tf.minimum(width,224.0),0.0)
    height=predictions[:,3]*224                   # we need width + x
    height=tf.maximum(tf.minimum(height,224.0),0.0) # we need height +y
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    condition=tf.less(threshhold,IoU)
    sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    return tf.reduce_mean(sum)

# loss function
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)


def resnet_block(inputs,num_filters,kernel_size,strides,activation='relu'):
    x=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(inputs)
    x=BatchNormalization()(x)
    if(activation):
        x=Activation('relu')(x)
    return x


def resnet18():
    inputs=Input((224,224,3))
    
    # conv1
    x=resnet_block(inputs,64,[7,7],2)

    # conv2
    x=MaxPooling2D([3,3],2,'same')(x)
    for i in range(2):
        a=resnet_block(x,64,[3,3],1)
        b=resnet_block(a,64,[3,3],1,activation=None)
        x=keras.layers.add([x,b])
        x=Activation('relu')(x)
    
    # conv3
    a=resnet_block(x,128,[1,1],2)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=Conv2D(128,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,128,[3,3],1)
    b=resnet_block(a,128,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv4
    a=resnet_block(x,256,[1,1],2)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=Conv2D(256,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,256,[3,3],1)
    b=resnet_block(a,256,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    # conv5
    a=resnet_block(x,512,[1,1],2)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=Conv2D(512,kernel_size=[1,1],strides=2,padding='same',kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(x)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    a=resnet_block(x,512,[3,3],1)
    b=resnet_block(a,512,[3,3],1,activation=None)
    x=keras.layers.add([x,b])
    x=Activation('relu')(x)

    x=AveragePooling2D(pool_size=7,data_format="channels_last")(x)
    # out:1*1*512

    y=Flatten()(x)
    # out:512
    y=Dense(1000,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    outputs=Dense(4,kernel_initializer='he_normal',kernel_regularizer=l2(1e-3))(y)
    
    model=Model(inputs=inputs,outputs=outputs)
    return model
  
  
keras.losses.smooth_l1_loss = smooth_l1_loss

# If you are training the model for the first time
model = resnet18()



#if you are re-training a pretrained model
# model = load_model('gdrive/My Drive/model.h5',custom_objects = {'smooth_l1_loss': smooth_l1_loss,'my_metric':my_metric})




model.compile(loss="smooth_l1_loss",optimizer=Adam(),metrics=[my_metric])

# model.summary()

def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5

# defining a custom data generator            
def generator(batch_size):
    
    counter=0
    epoch_counter = 0
    data_size = 1000     # size of data loaded when you do pickle.load
    
    f=open("final_id_to_data","rb")
    index=[i for i in range(data_size)]
    random.shuffle(index)
    X_data = pickle.load(f) 
    X_data = X_data[index]
    f1=open("final_id_to_box","rb")
    y_data = pickle.load(f1) 
    y_data = y_data[index]
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size

    while 1:

        X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        counter += 1
        epoch_counter += 1
        yield X_batch,y_batch

        #restart counter to yeild data in the next epoch as well
        if epoch_counter >= 200 :
            epoch_counter = 0;
            counter = 0;
            f=open("final_id_to_data","rb")
            index=[i for i in range(data_size)]
            random.shuffle(index)
            X_data = pickle.load(f) 
            X_data = X_data[index]
            f1=open("final_id_to_box","rb")
            y_data = pickle.load(f1) 
            y_data = y_data[index]
            
        if counter >= number_of_batches:
            X_data = pickle.load(f)
            counter = 0
            X_data = X_data[index]
            y_data = pickle.load(f1)
            y_data = y_data[index]
            
            
            
data_test,box_test=getdata()
            
            
def val_generator(X_data, y_data, batch_size):

    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0

    while 1:

        X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
        counter += 1
        yield X_batch,y_batch

        #restart counter to yeild data in the next epoch as well
        if counter >= number_of_batches:
            counter = 0




            
lr_scheduler=LearningRateScheduler(lr_sch)
lr_reducer=ReduceLROnPlateau(monitor='val_my_metric',factor=0.2,patience=5,mode='max',min_lr=1e-3)

checkpoint=ModelCheckpoint('model.h5',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

# model_details=model.fit(data_train,box_train,batch_size=128,epochs=100,shuffle=True,validation_split=0.1,callbacks=[lr_scheduler,lr_reducer,checkpoint],verbose=1)
# model_details=model.fit_generator(generator(batch_size=100),steps_per_epoch = 200,epochs=100,verbose=1,callbacks=[lr_scheduler,lr_reducer,checkpoint])

model_details=model.fit_generator(generator(batch_size=100),steps_per_epoch = 200,epochs=200,verbose=1,callbacks=[lr_scheduler,lr_reducer,checkpoint],validation_data=val_generator(data_test,box_test,100), validation_steps=(data_test.shape[0])/100)

model.save('gdrive/My Drive/model.h5')

scores=model.evaluate(data_test,box_test,verbose=1)
print('Test loss : ',scores[0])
print('Test accuracy : ',scores[1])

plot_model(model_details)


#GET_DATA_TEST

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
import random
import math
import keras.losses
from PIL import Image
import numpy as np
import pickle

import tensorflow as tf

plt.switch_backend('agg')
#getdata
def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

def Normalize(image,mean,std):
    for channel in range(3):
        image[:,:,channel]=(image[:,:,channel]-mean[channel])/std[channel]
    return image

def Imagegen(start,batchsize):
    id_to_data={}
    id_to_size={}
    j=1
    i=start
    
    with open("{/content}/datasets/f2016759/testid/testid.txt") as f:
        lines=f.read().splitlines()
        for line in lines:
            if (j>=i) and (j<=i+batchsize):
                id,path=line.split(" ",1)
                image=Image.open("{/content}/datasets/f2016759/test-images/test_images/"+path).convert('RGB')
                #print(np.array(image,dtype=np.float32).shape[0:2])
                id_to_size[int(id)]=np.array(image,dtype=np.float32).shape[0:2]                
                image=image.resize((224,224))
                image=np.array(image,dtype=np.float32)
                image=image/255
                image=Normalize(image,[0.485,0.456,0.406],[0.229,0.224,0.225])
                id_to_data[int(id)]=image                
                
            j=j+1
    id_to_data=np.array(list(id_to_data.values()))
    id_to_size=np.array(list(id_to_size.values()))
    f=open("id_to_data","wb+")
    pickle.dump(id_to_data,f,protocol=4)
    f=open("id_to_size","wb+")
    pickle.dump(id_to_size,f,protocol=4)
    return id_to_data
    
i=1
batchsize=1000
# total test images = 24045
for i in range(1,25000,batchsize):
    print(i)
    id_to_data= Imagegen(i,batchsize)
    f=open("id_to_data","wb+")
    pickle.dump(id_to_data,f,protocol=4)
    with open("gdrive/My Drive/final_id_to_data_test_rem", "ab") as myfile, open("id_to_data", "rb") as file2:
      myfile.write(file2.read())
    

#TEST

In [0]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
import random
import math
import keras.losses
import tensorflow as tf

plt.switch_backend('agg')


def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

def Normalize(image,mean,std):
    for channel in range(3):
        image[:,:,channel]=(image[:,:,channel]-mean[channel])/std[channel]
    return image

def my_metric(labels,predictions):
    threshhold=0.75
    x=predictions[:,0]*224
    x=tf.maximum(tf.minimum(x,224.0),0.0)
    y=predictions[:,1]*224
    y=tf.maximum(tf.minimum(y,224.0),0.0)
    width=predictions[:,2]*224                                         # we need width + x
    width=tf.maximum(tf.minimum(width,224.0),0.0)
    height=predictions[:,3]*224                 
    height=tf.maximum(tf.minimum(height,224.0),0.0) # we need height +y
    label_x=labels[:,0]
    label_y=labels[:,1]
    label_width=labels[:,2]
    label_height=labels[:,3]
    a1=tf.multiply(width,height)
    a2=tf.multiply(label_width,label_height)
    x1=tf.maximum(x,label_x)
    y1=tf.maximum(y,label_y)
    x2=tf.minimum(x+width,label_x+label_width)
    y2=tf.minimum(y+height,label_y+label_height)
    IoU=tf.abs(tf.multiply((x1-x2),(y1-y2)))/(a1+a2-tf.abs(tf.multiply((x1-x2),(y1-y2))))
    condition=tf.less(threshhold,IoU)
    sum=tf.where(condition,tf.ones(tf.shape(condition)),tf.zeros(tf.shape(condition)))
    return tf.reduce_mean(sum)

def smooth_l1_loss(true_box,pred_box):
    loss=0.0
    for i in range(4):
        residual=tf.abs(true_box[:,i]-pred_box[:,i]*224)
        condition=tf.less(residual,1.0)
        small_res=0.5*tf.square(residual)
        large_res=residual-0.5
        loss=loss+tf.where(condition,small_res,large_res)
    return tf.reduce_mean(loss)

  
k=1
f=open("gdrive/My Drive/final_id_to_data_test","rb")


with open("gdrive/My Drive/testid.txt") as g:
        lines=g.read().splitlines()
result_file =  open("gdrive/My Drive/final.csv", "a")

#Total test images = 24045

for j in range (0,25):
    data = pickle.load(f)
    if j==24:
      index = [i for i in range(45)]
    else:
      index = [i for i in range(1000)]
    model=keras.models.load_model("gdrive/My Drive/model.h5",custom_objects = {'smooth_l1_loss': smooth_l1_loss,'my_metric':my_metric})
#keras.losses.smooth_l1_loss = smooth_l1_loss
    result=model.predict(data[index,:,:,:])
    mean=[0.485,0.456,0.406]
    std=[0.229,0.224,0.225]
    for i in index:
      print("Predicting "+str(k)+"th image.")
      image=data[i]
      prediction=result[i]
      for channel in range(3):
          image[:,:,channel]=image[:,:,channel]*std[channel]+mean[channel]

      image=image*255
      image=image.astype(np.uint8)
      plt.imshow(image)


      plt.gca().add_patch(plt.Rectangle((prediction[0]*224,prediction[1]*224),prediction[2]*224,prediction[3]*224,fill=False,edgecolor='green',linewidth=2,alpha=0.5))
      plt.show()
      plt.savefig("./prediction/"+str(k)+".png")
      plt.cla()


      x1 = ((prediction[0])*640)
      x2 = (x1+(prediction[2])*640)
      y1 = ((prediction[1])*480)
      y2 = (y1+ (prediction[3])*480)
      id,path=lines[k-1].split(" ",1)  
      k=k+1
      result_file.write(path.rstrip()+ "," + str(x1)+ ","+str(x2) + "," + str(y1) + "," + str(y2) + "\n")